In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import nilearn

from nilearn import plotting
from nilearn import datasets
from nilearn import image
from nilearn.image import mean_img

# Let us use a Nifti file that is shipped with nilearn
from nilearn.datasets import MNI152_FILE_PATH

/home/brunomiguel/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
os.getcwd()

'/home/brunomiguel/Documents/GitHub/MVPA-speech_project/MVPA-project-admin'

In [ ]:
# Set variables
ROOT_DATA='/Users/home/Documents/BIDS/'
SUB='sub-0001'
SES='ses-001'

# data folder
data_path=os.path.join(root_data,sub,ses)
print('The data is in this folder - ' + data_path)

# project folder
ROOT_PROJECT="/Users/home/Documents/GitHub/MVPA-speech_project"

# Structural data.

## Check registration using FSL BET (brain extraction) + FEAT

In [ ]:
t1w_file_path=data_path+'/anat/sub-0001_ses-001_run-01_T1w.nii.gz'
plotting.plot_img(t1w_file_path)

In [ ]:
t1w_brain_file_path='/Users/home/Documents/BIDS/sub-0001/ses-001/anat/sub-0001_ses-001_run-01_T1w_brain.nii.gz'

plotting.plot_img(t1w_brain_file_path)

In [ ]:
t1w_prepro_file_path='/Users/home/Documents/BIDS/sub-0001/ses-001/run-1.feat/reg/highres2standard.nii.gz'

cut_coords=(0, 0, 0)
plotting.plot_img(t1w_prepro_file_path, cut_coords)

In [ ]:
# (0,0,0) no standard MNI (mas tb em Talairach) aponta sempre para uma estrutura anatómica chamada de comissura anterior.
plotting.plot_img(datasets.MNI152_FILE_PATH, cut_coords)

# Functional data.
## Check preprocessing using FSL FEAT

In [ ]:
func_data_preproc='/Users/home/Documents/BIDS/sub-0001/ses-001/run-1.feat/filtered_func_data.nii.gz'

mean_fmri_img=mean_img(func_data_preproc)
plotting.plot_img(mean_fmri_img, cut_coords)

In [ ]:
func_data_preproc_2high='/Users/home/Documents/BIDS/sub-0001/ses-001/run-1.feat/reg/example_func2standard.nii.gz'

mean_fmri_img_2high=mean_img(func_data_preproc_2high)

print(image.load_img(func_data_preproc_2high).shape)

plotting.plot_img(mean_fmri_img_2high, cut_coords)

In [ ]:
func_data_preproc2standard='/Users/home/Documents/BIDS/sub-0001/ses-001/run-1.feat/filtered_func_data2standard.nii.gz'
print(image.load_img(func_data_preproc2standard).shape)

mean_fmri_img_2standard=mean_img(func_data_preproc2standard)
plotting.plot_img(mean_fmri_img_2standard, cut_coords)

## Look at the events and design matrix.

In [ ]:
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix

tr = 2.5  # repetition time is ? second
n_scans = 222  # the acquisition comprises ?? scans
frame_times = np.arange(n_scans) * tr  # here are the correspoding frame times

# load events.tsv
events_PATH='/Users/home/Documents/GitHub/MVPA-speech_project/convert-bids-admin/DOCKER/events.tsv'

events_df = pd.read_csv(events_PATH, sep='\t', na_values="n/a")
print(events_df.head())

print(frame_times[:5])


In [ ]:
# load events.tsv
events_PATH='/Users/home/Documents/GitHub/MVPA-speech_project/convert-bids-admin/DOCKER/betaseries_fonologico.tsv'

events_df = pd.read_csv(events_PATH, sep='\t', na_values="n/a")
print(events_df.head(10))



In [ ]:
hrf_model='spm'
design_matrix = make_first_level_design_matrix(frame_times, events_df,
                                    drift_model='polynomial', drift_order=3,
                                    hrf_model=hrf_model)

plot_design_matrix(design_matrix)

**Statistical analysis - 1st level**

In [ ]:
contrast_matrix = np.eye(design_matrix.shape[1])
basic_contrasts = dict([(column, contrast_matrix[i])
                        for i, column in enumerate(design_matrix.columns)])


In [ ]:
from nilearn.glm.first_level import FirstLevelModel

print('Fitting a GLM')
fmri_glm = FirstLevelModel()
fmri_glm.fit(func_data_preproc2standard, design_matrices=design_matrix)

In [ ]:
z_maps=[]
for contrast in basic_contrasts:
    z_maps.append(fmri_glm.compute_contrast(basic_contrasts[contrast], output_type='z_score'))

In [ ]:
# plot the contrasts 
# the display is overlayed on the mean fMRI image
# a threshold of 3.0 is used, more sophisticated choices are possible
plotting.plot_glass_brain(
    z_maps[10], threshold=3.0, title='test')
plotting.show()



In [ ]:
len(z_maps)

# Decoding


In [ ]:
# brain_mask = datasets.load_mni152_brain_mask()

from nilearn.datasets import fetch_icbm152_brain_gm_mask
brain_mask = fetch_icbm152_brain_gm_mask()


dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm') 
atlas_filename = dataset.maps

frontal_mask = mean_img([math_img('img == %d'  %i, img=atlas_filename) for i in [48]])
plotting.view_img(frontal_mask)


In [ ]:
from nilearn.decoding import Decoder 

decoder = Decoder(estimator='svc', mask=frontal_mask) 

In [ ]:
plotting.plot_roi(frontal_mask, bg_img=mean_fmri_img_2standard,
                  cmap='Paired')

In [ ]:
# input are z_maps

# load events.tsv
events_PATH='/Users/home/Documents/GitHub/MVPA-speech_project/convert-bids-admin/DOCKER/events_epoch_fonologico.tsv'

events_df = pd.read_csv(events_PATH, sep='\t', na_values="n/a")
print(events_df.head(10))

conditions=events_df['trial_type'][:216]


In [ ]:
len(conditions)
print(conditions)

In [ ]:
from nilearn.image import index_img
trainset = index_img(z_maps, slice(0, -54))
trainset.shape

In [ ]:
decoder.fit(trainset, conditions[:-50]) 

In [ ]:
print(decoder.cv_scores_) 

In [ ]:
testset = index_img(z_maps, slice(-54,-4))
testset.shape

In [ ]:
prediction=decoder.predict(testset)

In [ ]:
print((prediction == conditions[-50:]).sum() / float(len(conditions[-50:])))

In [ ]:
plotting.view_img(
    decoder.coef_img_['Task '], bg_img=t1w_prepro_file_path,
    title="SVM weights", dim=-1
)

# dummy decoder - como avaliar a qualidade de um classificador?

In [ ]:
dummy_decoder = Decoder(estimator='dummy_classifier', mask=brain_mask)

dummy_decoder.fit(trainset, conditions[:-50]) 

dummy_prediction=dummy_decoder.predict(trainset)


In [ ]:
dummy_decoder.dummy_output_

# Functional connectivity

In [ ]:
dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_filename = dataset.maps
labels = dataset.labels

#yeo = datasets.fetch_atlas_yeo_2011()
#atlas_filename=yeo['thick_17']

In [ ]:
#yeo.description


In [ ]:
coordinates = plotting.find_parcellation_cut_coords(atlas_filename)
plotting.plot_roi(atlas_filename)

In [ ]:
coordinates


In [ ]:
from nilearn.input_data import NiftiLabelsMasker
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True)

In [ ]:
from nilearn.image import index_img

baseline_imgs = index_img(func_data_preproc2standard, baselineidxs)
task_imgs = index_img(func_data_preproc2standard, taskidxs)

In [ ]:
baseline_series = masker.fit_transform(baseline_imgs)
task_series = masker.fit_transform(task_imgs)

In [ ]:
print(baseline_series.shape)
print(task_series.shape)

In [ ]:
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix_baseline = correlation_measure.fit_transform([baseline_series])[0]

correlation_matrix_task = correlation_measure.fit_transform([task_series])[0]

# Plot the correlation matrix
import numpy as np
from nilearn import plotting
# Make a large figure
# Mask the main diagonal for visualization:
np.fill_diagonal(correlation_matrix_baseline, 0)
# The labels we have start with the background (0), hence we skip the
# first label
# matrices are ordered for block-like representation
plotting.plot_matrix(correlation_matrix_baseline, figure=(10, 8), labels=labels[1:],
                     vmax=0.8, vmin=-0.8, reorder=True)

# Make a large figure
# Mask the main diagonal for visualization:
np.fill_diagonal(correlation_matrix_task, 0)
# The labels we have start with the background (0), hence we skip the
# first label
# matrices are ordered for block-like representation
plotting.plot_matrix(correlation_matrix_task, figure=(10, 8), labels=labels[1:],
                     vmax=0.8, vmin=-0.8, reorder=True)

In [ ]:
plotting.plot_connectome(correlation_matrix_baseline, coordinates,
                         title='baseline corr',
                         edge_threshold="90%",edge_cmap="copper", colorbar="true")

plotting.plot_connectome(correlation_matrix_task, coordinates,
                         title='task corr',
                         edge_threshold="90%",edge_cmap="copper", colorbar="true")
